# 밸런스 게임 챗봇


##### 밸런스 게임 주제를 전달하고 이를 챗봇과 토론해볼 수 있도록 합니다.....(제가 할 수 있을까요...?)

1. 챗봇에게 밸런스 게임 주제를 출력하도록 합니다.
2. 밸런스 게임 주제가 선정되었으면 내가 답변을 하도록 하고 챗봇이 나의 생각을 바꾸기 위해 딴지를 걸게 합니다.

In [15]:
from dotenv import load_dotenv
import os

load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

In [14]:
from gtts import gTTS
import speech_recognition as sr

1. 주제를 만드는 함수

In [18]:
from openai import OpenAI

def make_thema(query, temperature=0.3):
    client = OpenAI()

    system_instruction = '''
    당신은 금요일에 퇴근하고 친구들을 만나 맛있는 저녁을 먹고 술을 적당히 마셔 신난 상태입니다. 
    술자리에서 다같이 재미있게 얘기할 만한 연애 밸런스 게임 주제 세 개를 선정해주세요.

    ### 지시사항 ###
    - 토론이 가능한 주제를 선정할 것
    - 성인이니까 선정적인 주제도 하나 정도는 출력할 것
    - 선택하기 어렵고 애매한 주제를 선택할 것
    - 여사친/남사친 문제를 한 개 이상 포함할 것

    ### 출력 형식 ###
    주제 1. 선택지 vs 선택지
    주제 2. 선택지 vs 선택지

    어떤 것에 대해 이야기해볼까요?
    '''
    user_message = f'''
    {query}
    '''

    response = client.chat.completions.create(

        model='gpt-4o',
        messages=[{
            'role': 'system',
            'content': [{
                'type': 'text',
                'text': system_instruction
            }]

        },
        {
            'role': 'user',
            'content': [{
                'type': 'text',
                'text': user_message

            }]

            
        }],
        response_format={
            'type':'text'
        },
        stream=True,
        temperature=temperature,
        max_tokens=2048,
        top_p = 1,
        frequency_penalty=0,
        presence_penalty=0
    )

    return response.choices[0].message.content

In [ ]:
# !pip install pyglet

   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------------------------------------- 1.0/1.0 MB 5.4 MB/s eta 0:00:00


In [12]:
from gtts import gTTS
import pyglet

first_input= '안녕하세요! 어떤 것을 도와드릴까요?'
opening_tts = gTTS(
    text=first_input,
    lang='ko'
)
# opening_tts.save('opening_ment.mp3')

music = pyglet.media.load('opening_ment.mp3',streaming=False)
music.play()
print(first_input)

안녕하세요! 어떤 것을 도와드릴까요?


In [20]:
recognizer = sr.Recognizer()
while True:
    with sr.Microphone() as source:
        print(first_input)
        audio = recognizer.listen(source)
        txt = recognizer.recognize_google(audio, language='ko')
        
        if txt is not None:
            print(txt)
            make_thema(txt)
            break
        


안녕하세요! 어떤 것을 도와드릴까요?
안녕 재미있는 밸런스 게임 주제 몇 가지 선정해 줘


AttributeError: 'Stream' object has no attribute 'choices'

In [13]:
print(make_thema('안녕! 재미있는 밸런스 게임 주제 세 개를 선정해줘!'))


안녕하세요! 여기 재미있는 연애 밸런스 게임 주제 세 개를 준비했어요:

주제 1. 연인의 여사친/남사친과 단둘이 여행 가기 vs 연인의 여사친/남사친과 단둘이 술 마시기

주제 2. 연인이 과거에 한 번 바람핀 적이 있지만 지금은 완전히 신뢰할 수 있는 상태 vs 연인이 과거에 바람핀 적은 없지만 지금은 의심스러운 행동을 하는 상태

주제 3. 연인과의 첫날밤에 갑자기 부모님이 방문하기 vs 연인과의 첫날밤에 갑자기 직장 상사가 방문하기

어떤 것에 대해 이야기해볼까요?


In [ ]:
from openai import OpenAI

def make_thema(query, temperature=0.3):
    client = OpenAI()

    system_instruction = '''
    당신은 금요일에 퇴근하고 친구들을 만나 맛있는 저녁을 먹고 술을 적당히 마셔 신난 상태입니다. 
    술자리에서 다같이 재미있게 얘기할 만한 연애 밸런스 게임 주제 세 개를 선정해주세요.

    ### 지시사항 ###
    - 토론이 가능한 주제를 선정할 것.
    - 성인이니까 선정적인 주제도 하나 정도는 출력할 것.
    - 선택하기 어렵고 애매한 주제를 선택할 것

    
    ### 출력 형식 ###
    주제 1. 선택지 vs 선택지
    주제 2. 선택지 vs 선택지

    어떤 것에 대해 이야기해볼까요?
    '''
    user_message = f'''
    {query}
    '''

    response = client.chat.completions.create(

        model='gpt-4o',
        messages=[{
            'role': 'system',
            'content': [{
                'type': 'text',
                'text': system_instruction
            }]

        },
        {
            'role': 'user',
            'content': [{
                'type': 'text',
                'text': user_message

            }]

            
        }],
        response_format={
            'type':'text'
        },
        temperature=temperature,
        max_tokens=2048,
        top_p = 1,
        frequency_penalty=0,
        presence_penalty=0
    )

    return response.choices[0].message.content

In [ ]:
chat_log = [
    {'role': 'system', 'content' :['너는 상대가 의견을 바꿀 수 있도록 설득하는 챗봇이야']}
    {'role': 'user', 'content': }
]